In [403]:
#PIP INSTALLATION
print('---1---')
%pip install pandas
print('---2---')
%pip install seaborn
print('---3---')
%pip install scikit-learn
print('---4---')
%pip install catboost
print('---5---')
%pip install ipywidgets
print('---6---')
%pip install pandas openpyxl

---1---
---2---




[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
---3---



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
---4---


...


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
---6---



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [404]:
#FROM LIBRARIES
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier, Pool
from IPython.display import FileLink
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_curve, auc, classification_report
#IMPORT AS
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

In [405]:
file_path = 'Dataset/diabetic_data.csv'
df = pd.read_csv(file_path,keep_default_na=False)
df_clean = df.copy()
df_clean.head()


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


DATA CLEANING

In [406]:
#REMOVING DUPLICATE DATA'S
df_clean['patient_nbr'].value_counts()
df_clean = df_clean.drop_duplicates(subset=['patient_nbr'])

In [407]:
features_drop_list = ['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'medical_specialty', 'repaglinide', 'nateglinide', 'chlorpropamide', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone','metformin-pioglitazone', 'acetohexamide', 'tolbutamide']
df_clean.drop(features_drop_list, axis=1, inplace=True)

In [408]:
#start by setting all values containing E or V into 0 (as one category)
df_clean.loc[df_clean['diag_1'].str.contains('V',na=False,case=False), 'diag_1'] = 0
df_clean.loc[df_clean['diag_1'].str.contains('E',na=False,case=False), 'diag_1'] = 0
df_clean.loc[df_clean['diag_2'].str.contains('V',na=False,case=False), 'diag_2'] = 0
df_clean.loc[df_clean['diag_2'].str.contains('E',na=False,case=False), 'diag_2'] = 0
df_clean.loc[df_clean['diag_3'].str.contains('V',na=False,case=False), 'diag_3'] = 0
df_clean.loc[df_clean['diag_3'].str.contains('E',na=False,case=False), 'diag_3'] = 0

#setting all missing values into -1
df_clean['diag_1'] = df_clean['diag_1'].replace('?', -1)
df_clean['diag_2'] = df_clean['diag_2'].replace('?', -1)
df_clean['diag_3'] = df_clean['diag_3'].replace('?', -1)

#No all diag values can be converted into numeric values
df_clean['diag_1'] = df_clean['diag_1'].astype(float)
df_clean['diag_2'] = df_clean['diag_2'].astype(float)
df_clean['diag_3'] = df_clean['diag_3'].astype(float)

In [409]:
# Ensure 'diag_1', 'diag_2', 'diag_3' are numeric
df_clean[['diag_1', 'diag_2', 'diag_3']] = df_clean[['diag_1', 'diag_2', 'diag_3']].apply(pd.to_numeric, errors='coerce')

def categorize_diag(df_clean, col):
    df_clean.loc[(df_clean[col] >= 1) & (df_clean[col] < 140), col] = 1
    df_clean.loc[(df_clean[col] >= 140) & (df_clean[col] < 240), col] = 2
    df_clean.loc[(df_clean[col] >= 240) & (df_clean[col] < 280), col] = 3
    df_clean.loc[(df_clean[col] >= 280) & (df_clean[col] < 290), col] = 4
    df_clean.loc[(df_clean[col] >= 290) & (df_clean[col] < 320), col] = 5
    df_clean.loc[(df_clean[col] >= 320) & (df_clean[col] < 390), col] = 6
    df_clean.loc[(df_clean[col] >= 390) & (df_clean[col] < 460), col] = 7
    df_clean.loc[(df_clean[col] >= 460) & (df_clean[col] < 520), col] = 8
    df_clean.loc[(df_clean[col] >= 520) & (df_clean[col] < 580), col] = 9
    df_clean.loc[(df_clean[col] >= 580) & (df_clean[col] < 630), col] = 10
    df_clean.loc[(df_clean[col] >= 630) & (df_clean[col] < 680), col] = 11
    df_clean.loc[(df_clean[col] >= 680) & (df_clean[col] < 710), col] = 12
    df_clean.loc[(df_clean[col] >= 710) & (df_clean[col] < 740), col] = 13
    df_clean.loc[(df_clean[col] >= 740) & (df_clean[col] < 760), col] = 14
    df_clean.loc[(df_clean[col] >= 760) & (df_clean[col] < 780), col] = 15
    df_clean.loc[(df_clean[col] >= 780) & (df_clean[col] < 800), col] = 16
    df_clean.loc[(df_clean[col] >= 800) & (df_clean[col] < 1000), col] = 17
    df_clean.loc[df_clean[col] == -1, col] = 0  # Handle missing values
    
# Apply function to diagnosis columns
for col in ['diag_1', 'diag_2', 'diag_3']:
    categorize_diag(df_clean, col)

# Print the result
print(df_clean[['diag_1', 'diag_2', 'diag_3']].head())

   diag_1  diag_2  diag_3
0     3.0     0.0     0.0
1     3.0     3.0     3.0
2    11.0     3.0     0.0
3     1.0     3.0     7.0
4     2.0     2.0     3.0


In [410]:
df_clean.diag_2.unique()

array([ 0.,  3.,  2.,  7.,  8.,  4., 17., 13., 10.,  1.,  9., 12.,  6.,
       16.,  5., 11., 14.])

In [411]:
df_clean['race'] = df_clean['race'].replace('?', 'Other')
df_clean.race.value_counts()

race
Caucasian          53491
AfricanAmerican    12887
Other               3126
Hispanic            1517
Asian                497
Name: count, dtype: int64

In [412]:
df_clean['gender'] = df_clean['gender'].replace('Unknown/Invalid', 'Female')
df_clean.gender.value_counts()

gender
Female    38028
Male      33490
Name: count, dtype: int64

In [413]:
df_clean['gender'] = df_clean['gender'].replace('Male', 1)
df_clean['gender'] = df_clean['gender'].replace('Female', 0)
df_clean.gender.value_counts()

C:\Users\SnowPlay\AppData\Local\Temp\ipykernel_17584\2177177959.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['gender'] = df_clean['gender'].replace('Female', 0)


gender
0    38028
1    33490
Name: count, dtype: int64

In [414]:
age_replacements = {}
for i in range(0, 10):
    age_range = f'[{10*i}-{10*(i+1)})'
    midpoint = i*10 + 5
    age_replacements[age_range] = midpoint

df_clean['age'] = df_clean['age'].replace(age_replacements)

df_clean['age'].value_counts()

C:\Users\SnowPlay\AppData\Local\Temp\ipykernel_17584\856190937.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['age'] = df_clean['age'].replace(age_replacements)


age
75    18210
65    15960
55    12466
85    11589
45     6878
35     2699
95     1900
25     1127
15      535
5       154
Name: count, dtype: int64

-- Start Here --

max_glu_serum -> None, Norm, >200, >300

A1Cresult -> None, Norm, >7, >8

drug('metformin',glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin) -> None,Steady,Down,Up

change -> No, Ch

diabetesMed -> Yes, No

In [415]:
df_clean['readmitted']=df_clean['readmitted'].replace('NO', 0)
df_clean['readmitted']=df_clean['readmitted'].replace('>30', 1)
df_clean['readmitted']=df_clean['readmitted'].replace('<30', 2)
df_clean.readmitted.value_counts()

C:\Users\SnowPlay\AppData\Local\Temp\ipykernel_17584\3891844975.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['readmitted']=df_clean['readmitted'].replace('<30', 2)


readmitted
0    42985
1    22240
2     6293
Name: count, dtype: int64

In [416]:
df_clean = pd.concat([df_clean,pd.get_dummies(df_clean['race'], prefix='race')], axis=1).drop(['race'],axis=1)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71518 entries, 0 to 101765
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gender                    71518 non-null  int64  
 1   age                       71518 non-null  int64  
 2   admission_type_id         71518 non-null  int64  
 3   discharge_disposition_id  71518 non-null  int64  
 4   admission_source_id       71518 non-null  int64  
 5   time_in_hospital          71518 non-null  int64  
 6   num_lab_procedures        71518 non-null  int64  
 7   num_procedures            71518 non-null  int64  
 8   num_medications           71518 non-null  int64  
 9   number_outpatient         71518 non-null  int64  
 10  number_emergency          71518 non-null  int64  
 11  number_inpatient          71518 non-null  int64  
 12  diag_1                    71518 non-null  float64
 13  diag_2                    71518 non-null  float64
 14  diag_3    

In [417]:
race_columns = ['race_AfricanAmerican', 'race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Other']
df_clean[race_columns] = df_clean[race_columns].astype(int)
# Check the changes
print(df_clean[race_columns].head())

   race_AfricanAmerican  race_Asian  race_Caucasian  race_Hispanic  race_Other
0                     0           0               1              0           0
1                     0           0               1              0           0
2                     1           0               0              0           0
3                     0           0               1              0           0
4                     0           0               1              0           0


In [418]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71518 entries, 0 to 101765
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gender                    71518 non-null  int64  
 1   age                       71518 non-null  int64  
 2   admission_type_id         71518 non-null  int64  
 3   discharge_disposition_id  71518 non-null  int64  
 4   admission_source_id       71518 non-null  int64  
 5   time_in_hospital          71518 non-null  int64  
 6   num_lab_procedures        71518 non-null  int64  
 7   num_procedures            71518 non-null  int64  
 8   num_medications           71518 non-null  int64  
 9   number_outpatient         71518 non-null  int64  
 10  number_emergency          71518 non-null  int64  
 11  number_inpatient          71518 non-null  int64  
 12  diag_1                    71518 non-null  float64
 13  diag_2                    71518 non-null  float64
 14  diag_3    

In [419]:
print(list(df_clean))

['gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin', 'change', 'diabetesMed', 'readmitted', 'race_AfricanAmerican', 'race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Other']


In [420]:
X = df_clean.drop('readmitted', axis=1)  # Features
y = df_clean['readmitted']               # Target variable

In [421]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_features = [col for col in X.columns if X[col].dtype == 'object']


In [422]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store accuracy scores
accuracy_scores = []

# Perform Stratified K-Fold CV
for train_index, test_index in skf.split(X, y):
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # Create Pool objects for train and test sets
    train_pool = Pool(data=X_train_fold, label=y_train_fold, cat_features=cat_features)
    test_pool = Pool(data=X_test_fold, label=y_test_fold, cat_features=cat_features)

    # Initialize CatBoost model
    model = CatBoostClassifier(
        learning_rate=0.1,
        depth=6,
        iterations=500,
        l2_leaf_reg=3,
        border_count=32,
        bootstrap_type='Bernoulli',  # Avoid Bayesian bootstrap (does not support subsample)
        subsample=0.8,
        colsample_bylevel=0.8,
        verbose=100  # Suppress training logs
    )

    # Train using Pool objects
    model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50, verbose=0)

    # Predict on the test fold
    y_pred_fold = model.predict(X_test_fold)

    # Compute accuracy
    accuracy = accuracy_score(y_test_fold, y_pred_fold)
    accuracy_scores.append(accuracy)

# Print results
print(f"Mean Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Standard Deviation: {np.std(accuracy_scores):.4f}")

Mean Accuracy: 0.6198
Standard Deviation: 0.0020


In [423]:
# Make predictions
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.4f}")  # Print with 4 decimal places

Model Accuracy: 0.6394


In [424]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.93      0.77      8732
           1       0.55      0.22      0.31      4408
           2       0.81      0.02      0.04      1164

    accuracy                           0.64     14304
   macro avg       0.67      0.39      0.37     14304
weighted avg       0.63      0.64      0.57     14304



In [425]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[8162  568    2]
 [3443  962    3]
 [ 938  204   22]]
